# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [33]:
import os
import re

import pandas as pd

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
print(os.getcwd())

C:\Users\chziha\Data-Scientist-Udacity-Nanodegree\DSND_Term2\project_files\Disaster-Response-Pipelines-trials


In [4]:
# load data from database
engine = create_engine('sqlite:///C:\\Users\\chziha\\Data-Scientist-Udacity-Nanodegree\\DSND_Term2\\project_files\\Disaster-Response-Pipelines-trials\\trial.db')
df = pd.read_sql_table('MessageCategory', engine)
df.head() 

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [30]:
# Define feature and target variables
X = df['message']
y = df[df.columns.drop(['id', 'message', 'original', 'genre'])]

### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    '''
    Function to tokenize the text data
    
    Args:
        text: a string
        
    Returns:
        words: a list of tokenized words
    '''
    # Replace all urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text.replace(url, 'urlplaceholder')
    
    # Normalize the text
    pattern = r'[^a-zA-Z0-9]'
    text = re.sub(pattern, ' ', text.lower())
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Lemmatize the tokens
    lemmed = [WordNetLemmatizer().lemmatize(w).strip() for w in tokens]

    # Remove stop words    
    words = [w for w in lemmed if w not in stopwords.words('english')]
    
    return words

In [25]:
# Test the tokenize function
print(df.message[12])
print(tokenize(df.message[12]))

I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks
['thomassin', 'number', '32', 'area', 'named', 'pyron', 'would', 'like', 'water', 'thank', 'god', 'fine', 'desperately', 'need', 'water', 'thanks']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
pipeline = Pipeline([
    ('countVect', CountVectorizer(tokenizer=tokenize)),
    ('tfidfTrans', TfidfTransformer()),
    ('RFclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.3, random_state = 23)

# Train the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('countVect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

In [34]:
a = pipeline.predict(X_test)

In [35]:
type(a)

numpy.ndarray

In [37]:
a[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [40]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
14462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17754,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
16108,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
277,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14898,1,0,0,1,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,0,0
4535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7096,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
print(classification_report(y_test.iloc[0, :], a[0]))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        36
           1       0.00      0.00      0.00         0

    accuracy                           0.92        36
   macro avg       0.50      0.46      0.48        36
weighted avg       1.00      0.92      0.96        36



In [44]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], a[:, i]))

related
              precision    recall  f1-score   support

           0       0.69      0.37      0.48      4251
           1       0.82      0.95      0.88     13968
           2       0.60      0.02      0.04       133

    accuracy                           0.81     18352
   macro avg       0.70      0.45      0.47     18352
weighted avg       0.79      0.81      0.78     18352

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     15188
           1       0.83      0.46      0.60      3164

    accuracy                           0.89     18352
   macro avg       0.86      0.72      0.77     18352
weighted avg       0.89      0.89      0.88     18352

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18273
           1       0.00      0.00      0.00        79

    accuracy                           1.00     18352
   macro avg       0.50      0.50      0.50     1835

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18352

    accuracy                           1.00     18352
   macro avg       1.00      1.00      1.00     18352
weighted avg       1.00      1.00      1.00     18352

water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     17182
           1       0.90      0.33      0.48      1170

    accuracy                           0.95     18352
   macro avg       0.93      0.66      0.73     18352
weighted avg       0.95      0.95      0.94     18352

food
              precision    recall  f1-score   support

           0       0.95      0.98      0.97     16312
           1       0.82      0.62      0.70      2040

    accuracy                           0.94     18352
   macro avg       0.89      0.80      0.84     18352
weighted avg       0.94      0.94      0.94     18352

shelter
              precision    recall  f1-sco

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def report_metrics(y_true, y_pred):
    '''
    Function to display the classification report for each category in multi-class classification
    
    Args:
        y_true: 2d array-like, or label indicator array / sparse matrix for the true y values
        y_pred: 2d array-like, or label indicator array / sparse matrix for the predicted y values
        
    Returns:
        None
    '''
    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.